<a href="https://colab.research.google.com/github/Ana-AlonsoCanizares/AA_GRUPO3/blob/main/RANKING_PADEL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [39]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
df = pd.read_excel('/content/Grupo 4.xlsx')

In [37]:
# # Actualizamos la función para manejar resultados con tres números (sets)
def calcular_sets_por_jornada(resultado1, resultado2):
    # Descomponer los resultados en números
    try:
        puntos1 = [int(p.strip()) for p in resultado1.split('-') if p.strip().isdigit()]
        puntos2 = [int(p.strip()) for p in resultado2.split('-') if p.strip().isdigit()]
    except ValueError:
        # En caso de que no haya un resultado válido, regresamos 0 para ambos
        return 0, 0

    # Inicializar los puntos de set para cada pareja
    puntos_set_pareja1 = 0
    puntos_set_pareja2 = 0

    # Comparamos cada set de números y asignamos un punto al ganador de cada set
    for p1, p2 in zip(puntos1, puntos2):
        if p1 > p2:
            puntos_set_pareja1 += 1
        elif p2 > p1:
            puntos_set_pareja2 += 1

    return puntos_set_pareja1, puntos_set_pareja2

def gana_pierde(resultado1, resultado2):
    try:
        puntos1 = [int(p.strip()) for p in resultado1.split('-') if p.strip().isdigit()]
        puntos2 = [int(p.strip()) for p in resultado2.split('-') if p.strip().isdigit()]
    except ValueError:
        return 0, 0

    # Inicializar los puntos de set para cada pareja
    puntos_set_pareja1 = 0
    puntos_set_pareja2 = 0

    # Comparamos cada set de números y asignamos un punto al ganador de cada set
    for p1, p2 in zip(puntos1, puntos2):
        if p1 > p2:
            puntos_set_pareja1 += 1
        elif p2 > p1:
            puntos_set_pareja2 += 1

    # Determinar qué pareja gana y cuál pierde
    if puntos_set_pareja1 > puntos_set_pareja2:
        return 1, 0
    elif puntos_set_pareja2 > puntos_set_pareja1:
        return 0, 1
    else:
        return 0, 0


def juegos(resultado1, resultado2):
    try:
        puntos1 = [int(p.strip()) for p in resultado1.split('-') if p.strip().isdigit()]
        puntos2 = [int(p.strip()) for p in resultado2.split('-') if p.strip().isdigit()]
    except ValueError:
        return 0, 0

    # Inicializar los puntos de set para cada pareja
    juegos_pareja1 = 0
    juegos_pareja2 = 0

    # Comparamos cada set de números y asignamos un punto al ganador de cada set

    juegos_pareja1 = sum(puntos1)
    juegos_pareja2 = sum(puntos2)

    return juegos_pareja1, juegos_pareja2

In [46]:
# Ahora calcularemos los puntos para todas las jornadas siguiendo la lógica del usuario
# Primero, asignamos las parejas que se enfrentan en cada jornada
enfrentamientos = {
    'JORNADA 1': [(0, 3), (1, 4), (2, 5)],
    'JORNADA 2': [(0, 2), (1, 5), (3, 4)],
    'JORNADA 3': [(0, 1), (2, 3), (4, 5)],
    'JORNADA 4': [(0, 5), (1, 3), (2, 4)],
    'JORNADA 5': [(0, 4), (1, 2), (3, 5)],
}

# Calculamos los puntos de set para las parejas que jugaron entre sí en cada jornada
for jornada, parejas in enfrentamientos.items():
    puntos_jornada = []
    resultados_jornada = df[jornada].tolist()
    for pareja in parejas:
        puntos_jornada.append(calcular_sets_por_jornada(resultados_jornada[pareja[0]], resultados_jornada[pareja[1]]))

    # Asignamos los puntos a las parejas correspondientes en el DataFrame original
    for i, pareja in enumerate(parejas):
        df.loc[pareja[0], f'Sets {jornada}'] = puntos_jornada[i][0]
        df.loc[pareja[1], f'Sets {jornada}'] = puntos_jornada[i][1]
        df.loc[pareja[0], f'Sets perdidos {jornada}'] = puntos_jornada[i][1]
        df.loc[pareja[1], f'Sets perdidos {jornada}'] = puntos_jornada[i][0]



for jornada in enfrentamientos.keys():
    df[f'Gana {jornada}'] = None  # Inicializar las columnas con None
    for pareja in enfrentamientos[jornada]:
        jug1, jug2 = juegos(df.loc[pareja[0], jornada], df.loc[pareja[1], jornada])
        df.loc[pareja[0], f'Gana juegos {jornada}'] = jug1
        df.loc[pareja[1], f'Gana juegos {jornada}'] = jug2
        df.loc[pareja[0], f'Pierde juegos {jornada}'] = jug2
        df.loc[pareja[1], f'Pierde juegos {jornada}'] = jug1

# Mostrar las nuevas columnas del DataFrame
df[[f'Gana {j}' for j in enfrentamientos.keys()]]

for jornada in enfrentamientos.keys():
    df[f'Gana {jornada}'] = None  # Inicializar las columnas con None
    for pareja in enfrentamientos[jornada]:
        ganador, perdedor = gana_pierde(df.loc[pareja[0], jornada], df.loc[pareja[1], jornada])
        df.loc[pareja[0], f'Gana {jornada}'] = ganador
        df.loc[pareja[1], f'Gana {jornada}'] = perdedor
        df.loc[pareja[0], f'Pierde {jornada}'] = perdedor
        df.loc[pareja[1], f'Pierde {jornada}'] = ganador

df['PG'] = df['Gana JORNADA 1'] + df['Gana JORNADA 2'] + df['Gana JORNADA 3'] + df['Gana JORNADA 4'] + df['Gana JORNADA 5']
df['PP'] = df['Pierde JORNADA 1'] + df['Pierde JORNADA 2'] + df['Pierde JORNADA 3'] + df['Pierde JORNADA 4'] + df['Pierde JORNADA 5']
df['PJ'] = df['PG'] + df['PP']
df['PUNTOS'] = df['PG']*3 + df['PP']

df['JG'] = df['Gana juegos JORNADA 1'] + df['Gana juegos JORNADA 2'] + df['Gana juegos JORNADA 3'] + df['Gana juegos JORNADA 4'] + df['Gana juegos JORNADA 5']
df['JP'] = df['Pierde juegos JORNADA 1'] + df['Pierde juegos JORNADA 2'] + df['Pierde juegos JORNADA 3'] + df['Pierde juegos JORNADA 4'] + df['Pierde juegos JORNADA 5']
df['DIF JUEGOS'] = df['JG'] - df['JP']
# Crear la nueva columna 'Suma Jornadas 1-3'
df['SET GANA'] = df['Sets JORNADA 1'] + df['Sets JORNADA 2'] + df['Sets JORNADA 3'] + df['Sets JORNADA 4'] + df['Sets JORNADA 5']
df['SET PIERDE'] = df['Sets perdidos JORNADA 1'] + df['Sets perdidos JORNADA 2'] + df['Sets perdidos JORNADA 3'] + df['Sets perdidos JORNADA 4'] + df['Sets perdidos JORNADA 5']
df['DIF SET'] = df['SET GANA'] - df['SET PIERDE']

# df[['PAREJAS', 'JORNADA 1', 'JORNADA 2', 'JORNADA 3', 'JORNADA 4', 'JORNADA 5', 'PJ', 'PG', 'PP', 'PUNTOS','SET GANA', 'SET PIERDE', 'DIF SET']]

In [50]:
clasificacion = df[['PAREJAS','PJ', 'PG', 'PP', 'PUNTOS','SET GANA', 'SET PIERDE', 'DIF SET', 'JG', 'JP', 'DIF JUEGOS']]
clasificacion = clasificacion.sort_values(by='PUNTOS', ascending=False)

# Seleccionar columnas numéricas
cols_numericas = clasificacion.select_dtypes(include=[np.number]).columns

# Convertir columnas numéricas a int
clasificacion[cols_numericas] = clasificacion[cols_numericas].astype(int)
clasificacion

,PAREJAS,PJ,PG,PP,PUNTOS,SET GANA,SET PIERDE,DIF SET,JG,JP,DIF JUEGOS
0,ALEJANDRO - ANA,5.0,5,0,15.0,10,0,10,60,13,47
3,RAQUEL - JULIO,5.0,3,2,11.0,7,4,3,53,37,16
1,ERIC - MIRIAM,4.0,3,1,10.0,6,3,3,45,42,3
5,JOSE CARLOS - ELVIRA,4.0,2,2,8.0,4,4,0,29,34,-5
4,INES - MACARENA,5.0,1,4,7.0,2,8,-6,33,58,-25
2,DANIEL - ANA,5.0,0,5,5.0,0,10,-10,25,61,-36
